## Yol Belirleme
Bu arada Google Drive içinde çalışacağınız klasörü belirlemeniz gerekmektedir. Veri seti düzenleme veya model eğitimi için burayı yeniden çalıştırın.

In [ ]:
!pwd

In [ ]:
%cd /content/drive/MyDrive/Veri-Kazima/colab

In [ ]:
!pwd

## Gerekli Kütüphaneleri Yükleyin
Burada model eğitimi, model çalıştırma için buraya yeniden gelebilirsiniz.

In [ ]:
#İlgili kütüphanelerin Kullanımı
!pip install transformers torch peft datasets accelerate bitsandbytes

In [ ]:
#Bazı durumlarda PyTorch için uygun olan CUDA sürümüne göre kurulum yapmanız gerekebilir.
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

In [ ]:
#Bazen Bitsanbtyes kütüphanesinde sorun olabiliyor, kaldırmayı deneyin
!pip uninstall bitsandbytes -y

In [ ]:
#Bitsanbytes kurululumunu yeniden yapın. Daha sonra Google Colab'ı yeniden başlatın.
!pip install -U bitsandbytes

## Veri Seti Kullanma
Burada daha önce Selenium kullanılarak çeşitli internet sitelerinden kazınan CSV formatında veriler yer almaktadır. Bu veriler ALPACA formatında JSON olarak yeniden kaydedeceğiz. Kaggle ile indirdiğimiz veri setinin ismini ise değiştirin ve bunu yapın : Tum-Veri-Seti-Nisan2024-10.csv

Eğer veri setinin ismini kendiniz belirlemek isterseniz koddaki ilgili alana dosya yolunu tam olarak yazınız.

In [ ]:
#Veriyi indirin, burada sizin için daha öncesinde topladığım 114.000 satır veri vardır. Bu veriyi Google Colab içine ekleyin.
#https://www.kaggle.com/datasets/denizhanahin/tr-dataset114k

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("denizhanahin/tr-dataset114k")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd
import json

def csv_to_alpaca(csv_file_path, output_json_path):
    """
    CSV dosyasını Alpaca formatına dönüştürür ve JSON dosyasına kaydeder.

    Args:
        csv_file_path (str): CSV dosyasının yolu.
        output_json_path (str): Oluşturulacak JSON dosyasının yolu.
    """

    # CSV dosyasını pandas DataFrame'ine oku
    df = pd.read_csv(csv_file_path)

    # Alpaca formatına dönüştür
    alpaca_data = []
    for index, row in df.iterrows():
        alpaca_entry = {
            "instruction": row["Baslik"],  # Başlık bilgisini "instruction" alanına ekle
            "input": row["Ozet"],  # Özet bilgisini "input" alanına ekle
            "output": row["Icerik"]  # İçerik bilgisini "output" alanına ekle
        }
        alpaca_data.append(alpaca_entry)

    # JSON dosyasına kaydet
    with open(output_json_path, "w", encoding="utf-8") as f:
        json.dump(alpaca_data, f, ensure_ascii=False, indent=4)

# Örnek kullanım
csv_file_path = "/content/drive/MyDrive/Veri-Kazima/Model_Eğitim/Tum-Veri-Seti-Nisan2024-10.csv"  # CSV dosyanızın adı
output_json_path = "1alpaca_format.json"  # Oluşturulacak JSON dosyasının adı
csv_to_alpaca(csv_file_path, output_json_path)

## Model Eğitimi - GPU
Google Colab üzerinde kullandığınız GPU birimini kullanarak model eğitimi yapılacaktır.

In [ ]:
# ----------------------------------------------
# 0. Gerekli Kütüphaneler
# ----------------------------------------------
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from google.colab import drive
import torch
import os

# ----------------------------------------------
# 1. Google Drive Bağlantısı
# ----------------------------------------------
drive.mount('/content/drive')
CHECKPOINT_DIR = "/content/drive/MyDrive/Veri-Kazima/colab/gemma2_model_yeni" #Model verilerini kayıt edeceğiniz klasörü seçin.

In [ ]:
import os
os.environ["HUGGING_FACE_HUB_TOKEN"] = "senin token"  # Hugging Face üzerinden ince ayar için kullanacağımız modeli indirmek için Hugging Face tokenı gerekmektedir

In [ ]:
# ----------------------------------------------
# 2. Tokenizer ve Padding Token Ayarları
# ----------------------------------------------
model_name = "google/gemma-2b" #"meta-llama/Meta-Llama-3-8B" #burada ise sahip olduğunuz RAM ve GPU RAM miktarına göre seçin. Ben kısa süreli eğitim için düşük parametreli model seçtim.
tokenizer = AutoTokenizer.from_pretrained(model_name, token="senin token") # Hugging Face üzerinden ince ayar için kullanacağımız modeli indirmek için Hugging Face tokenı gerekmektedir

# Padding token ekleme (Mistral için gerekli)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Model ve tokenizer'ı yükle
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Yerel dizine kaydet
save_directory = "./deepseek-14B"  # Kaydedilecek klasör
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

In [ ]:
# ----------------------------------------------
# 3. Model Yükleme
# ----------------------------------------------
from transformers import BitsAndBytesConfig

# Quantization ayarlarını yapılandır
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit kantizasyon kullan
    bnb_4bit_quant_type="nf4",  # Kantizasyon tipi
    bnb_4bit_use_double_quant=True,  # Çift kantizasyon kullan
)


# Modeli yükle
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # Quantization ayarlarını geçir
    device_map="auto",
    torch_dtype=torch.float16
)
model.resize_token_embeddings(len(tokenizer))  # Yeni token için boyut ayarla

In [ ]:
# ----------------------------------------------
# 4. LoRA Konfigürasyonu #buradaki ayarları kendinize göre yapın, ben düşük seviyeli bir GPU ve iki saatlik Google Colab süresine göre ayarladım.
# ----------------------------------------------
peft_config = LoraConfig(
    r=16,#64
    lora_alpha=16,#64
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.001, #0.05
    bias="none",A
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)

In [ ]:
# ----------------------------------------------
# 5. Veri Hazırlama (Düzeltilmiş)
# ----------------------------------------------
def format_dataset(example):
    return {
        "instruction": example["instruction"],
        "input": example["input"],
        "output": example["output"]
    }

dataset = load_dataset("json", data_files="/content/drive/MyDrive/Veri-Kazima/colab/1alpaca_format.json")["train"]
dataset = dataset.map(format_dataset)

In [ ]:
# ----------------------------------------------
# 6. Özelleştirilmiş Data Collator (DÜZELTİLMİŞ)
# ----------------------------------------------
def custom_collator(batch):
    # Instruction + Input + Output formatını oluştur
    texts = [
        f"### Instruction: {x['instruction']}\n### Input: {x['input']}\n### Output: {x['output']}"
        for x in batch
    ]

    # Tokenize işlemi (LABELS dahil)
    inputs = tokenizer(
        texts,
        padding="longest",
        truncation=True,
        max_length=64,#256 #buradaki ayarları kendinize göre yapın, ben düşük seviyeli bir GPU ve iki saatlik Google Colab süresine göre ayarladım.
        return_tensors="pt"
    )

    # Labels = input_ids (Causal LM için)
    inputs["labels"] = inputs["input_ids"].clone()
    return inputs


# ----------------------------------------------
# 7. Eğitim Ayarları #buradaki ayarları kendinize göre yapın, ben düşük seviyeli bir GPU ve iki saatlik Google Colab süresine göre ayarladım.
# ----------------------------------------------
training_args = TrainingArguments(
    output_dir=  CHECKPOINT_DIR, #"./output", #çıkış yerine kendiniz seçin ama şimdi şu anlık değiştirmeyin
    per_device_train_batch_size=16,  # 2 -> 1 (bellek tasarrufu)
    gradient_accumulation_steps=1,   # 4 -> 8
    num_train_epochs=1,              # Toplam 3 epoch
    learning_rate=1e-4,              # 2e-4 -> 1e-4 (stabilite için)
    fp16=True,
    save_strategy="epoch",
    logging_steps=200,
    remove_unused_columns=False,
    report_to="none",
    optim="adamw_8bit"               # 8-bit optimizasyon
)

In [ ]:
# ----------------------------------------------
# 8. Trainer (DÜZELTİLMİŞ)
# ----------------------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=custom_collator
)

In [ ]:
# ----------------------------------------------
# 9. Eğitimi Başlat
# ----------------------------------------------
try:
    print("Checkpoint aranıyor...")
    trainer.train(resume_from_checkpoint=True)
except Exception as e:
    print(f"Hata: {str(e)[:200]}...\nYeni eğitim başlatılıyor...")
    trainer.train()

# ----------------------------------------------
# 10. Modeli Kaydet
# ----------------------------------------------
model.save_pretrained(f"{CHECKPOINT_DIR}/final_model")
tokenizer.save_pretrained(f"{CHECKPOINT_DIR}/final_model")

## PEFT ve LORA ile Ana Modeli Birleştirme
Yukarıdaki model eğitiminde LORA ve PEFT ile ince ayar yaptık. Burada ise bize bazı dosyalar oluşturdu. Şimdi ise model eğitimi için kullandığımız ana model ile eğitim sonuçlarını birleştirmemiz gerekmektedir. Bu sayede ince ayar yaptığımız modeli rahatça kullanabileceğiz.

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# ----------------------------------------------
# 1. Ana Model ve Adapter Yolları
# ----------------------------------------------
base_model_name = "google/gemma-2b"  # Orijinal model adı
peft_model_path = "/content/drive/MyDrive/Veri-Kazima/colab/output/checkpoint-7114" #modeli kayıt ettiğiniz yer

# ----------------------------------------------
# 2. Ana Modeli ve Tokenizer'ı Yükle
# ----------------------------------------------
tokenizer = AutoTokenizer.from_pretrained(base_model_name,token="senin token")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

# ----------------------------------------------
# 3. PEFT Adapter'ını Yükle
# ----------------------------------------------
model = PeftModel.from_pretrained(
    base_model,
    peft_model_path,
    adapter_name="default",
    device_map="auto"
)

# ----------------------------------------------
# 4. Modeli Birleştir ve Kullan
# ----------------------------------------------
model = model.merge_and_unload()  # Adapter'ı ana modele entegre et
# Eğitim sonunda kaydetme
model.save_pretrained("/content/drive/MyDrive/Veri-Kazima/colab/birleştirilmiş_model")
tokenizer.save_pretrained("/content/drive/MyDrive/Veri-Kazima/colab/birleştirilmiş_model")

## Model Eğitimini Kaydetme
Model eğitimi bittiğinde tüm veriler cihazınızda kayıtlı olacaktır. Bu verileri Hugging Face içine yükleyip çalıştırabilirsiniz.

In [ ]:
from huggingface_hub import login
login(token="senin token")

In [ ]:
from huggingface_hub import HfApi

# Repo adı (örnek: kullanıcı_adı/model_adı)
repo_name = "denizzhansahin/gemma2bDeneme"

# Modeli yükle
api = HfApi()
#api.create_repo(repo_id=repo_name, repo_type="model", private=True)  # Özel repo, oluşturmak için yorum satırını kullanma, güncellemek için yorum satırını kullan
api.upload_folder(
    folder_path="/content/drive/MyDrive/Veri-Kazima/colab/birleştirilmiş_model",
    repo_id=repo_name,
    repo_type="model"
)

## Model Çalıştırma
Artık ince ayar yaptığımız ve gerekli düzenlemeleri yaptığımız modelimizi çalıştırabiliriz. Daha önce Hugging Face üzerine yüklediğimiz modeli çalıştıralım.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("denizzhansahin/gemma2bDeneme")
model = AutoModelForCausalLM.from_pretrained("denizzhansahin/gemma2bDeneme")

input_text = "Fenerbahçe"
input_ids = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))